In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.1'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
!pip install findspark
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [339 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packa

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-03-21 16:06:02--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.01MB/s    in 0.2s    

2021-03-21 16:06:02 (6.01 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

### Create DataFrames to match tables

In [10]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [6]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   13780696|             1|
|   48489393|             8|
|   43380682|             1|
|   17133428|             1|
|    9731896|             2|
|    1797768|             1|
|   27446622|             1|
|     120706|             3|
|   15915947|             2|
|   37520920|             1|
|   43920023|             9|
|   14309190|             8|
|   22316505|             1|
|   43949054|             2|
|   21102747|             1|
|   19021463|             1|
|   16075459|             8|
|   51475782|             2|
|   11555870|             5|
|   35471383|             3|
+-----------+--------------+
only showing top 20 rows



In [7]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B01427DDXM|Zoozie LA Women's...|
|B013TPITFA|Tabeez Women's Pl...|
|B013FU4Y9E|Ekouaer Women's U...|
|B013FNK6PC|Ezcosplay Women O...|
|B013FACXVK|Womens Two Piece ...|
|B013DHKQ3W|OURS Women's 3 4 ...|
|B0131D4NKU|Genie Women's Jeg...|
|B0131CA9K4|BeWild Brand - Ju...|
|B0130ZDFTE|Flora&Fred Women'...|
|B012Y665WY|AW Fashion's I Ne...|
|B012X0G1KW|KAMACO Women's Se...|
|B012VOPU84|iLoveSIA Women's ...|
|B012UUEWZG|Harley-Davidson W...|
|B012LQU4CE|Women's Laced & L...|
|B012ERJSKO|Amoin Style Women...|
|B012ADLW92|Cocoship Retro 50...|
|B012980QUY|Sanwood® Cute Bab...|
|B0125V8SZK|OURS Women's Comf...|
|B0125M6TZ0|Modern Kiwi Solid...|
|B01254G39K|Alberto Cardinali...|
+----------+--------------------+
only showing top 20 rows



In [8]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
from pyspark.sql.functions import to_date
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1KKOXHNI8MSXU|   32158956|B01KL6O72Y|      24485154| 2013-01-14|
|R26SP2OPDK4HT7|    2714559|B01ID3ZS5W|     363128556| 2014-03-04|
| RWQEDYAX373I1|   12608825|B01I497BGY|     811958549| 2015-07-12|
|R231YI7R4GPF6J|   25482800|B01HDXFZK6|     692205728| 2015-06-03|
|R3KO3W45DD0L1K|    9310286|B01G6MBEBY|     431150422| 2015-06-12|
|R1C4QH63NFL5NJ|   26631939|B01FWRXN0Y|     366144407| 2014-04-22|
|R2GP65O1U9N7BP|   48785098|B01EXNH1HE|     786052021| 2015-07-28|
|R3O29CT5MQQ3XQ|   39548589|B01E7OL09O|     108920964| 2015-07-10|
|R1ZECD2AA8QFF6|   29355866|B01DXHX81O|     317132458| 2015-08-09|
|R2S79GCF6J89OA|   27477484|B01DDULIJK|     110598191| 2014-05-24|
|R1CBCUGNP37MPK|   17685865|B01BOKOL4A|     121408017| 2015-06-10|
|R3NU4WUR5ZNV1V|   19787539|B01B3Q4Q0O|     134506082| 2013-09

In [9]:
# Create the vine_table. DataFrame
from pyspark.sql.types import IntegerType
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df = vine_df.withColumn("star_rating", vine_df["star_rating"].cast(IntegerType()))
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1KKOXHNI8MSXU|          4|            0|          0|   N|                Y|
|R26SP2OPDK4HT7|          5|            1|          2|   N|                Y|
| RWQEDYAX373I1|          5|            0|          0|   N|                Y|
|R231YI7R4GPF6J|          5|            0|          0|   N|                Y|
|R3KO3W45DD0L1K|          5|            0|          0|   N|                Y|
|R1C4QH63NFL5NJ|          5|            0|          0|   N|                Y|
|R2GP65O1U9N7BP|          5|            0|          0|   N|                Y|
|R3O29CT5MQQ3XQ|          4|            0|          0|   N|                Y|
|R1ZECD2AA8QFF6|          5|            0|          0|   N|                Y|
|R2S79GCF6J89OA|          3|            0|          0|   N|     

In [11]:
#filter the dataset for total_votes being greater than or equal to 20
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vote_count = df.filter("total_votes >= 20")

In [12]:
#create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
new_table = vote_count.filter(vote_count["helpful_votes"]/vote_count["total_votes"]>=0.5)
new_table.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   17162238|R35PT06NWP7LDP|B0140UFQKE|     555753462|Women Padded Spor...|         Apparel|          5|           30|         32|   N|                N|This is an extrem...|This is an extrem...| 2015-08-28|
|         US|     453792| RQW4AFOG9MR4Z|B013QTWRAC|     254612789|UONBOX Celebrity ...|         Apparel|          5|    

In [13]:

#Create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
new_table.filter(new_table["vine"] == "Y").show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51744990| R6U9701C3BGO6|B00XL2X3U8|     139556466|Wrangler Authenti...|         Apparel|          3|          139|        147|   Y|                N|        Comfort-Lite|After some debate...| 2015-08-31|
|         US|   35360512|R1XK3ALB45D7N4|B00XKYRLEQ|     243221870|Wrangler Authenti...|         Apparel|          5|    

In [14]:
# The total number of reviews, the number of 5-star reviews, and the percentage 5-star reviews are calculated for all Vine and non-Vine reviews (15 pt)
five_star = new_table.filter(new_table["star_rating"]== 5)
five_star.cache()

DataFrame[marketplace: string, customer_id: int, review_id: string, product_id: string, product_parent: int, product_title: string, product_category: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]

In [15]:
five_star.count()

23748

In [16]:

# Total reviews
new_table.count()

45421

In [17]:

# Percentage of five star reviews
five_star.count() / new_table.count()

0.5228418572906806

In [18]:
# Percentage of paid reviews
five_star.filter(five_star["verified_purchase"] == "Y").count()/new_table.filter(new_table["verified_purchase"]== "Y").count()

0.5291599198616791

In [19]:
# Percentage of unpaid reviews
five_star.filter(five_star["verified_purchase"] == "N").count()/new_table.filter(new_table["verified_purchase"]== "N").count()

0.4972172751558326